In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
import nltk
nltk.download('stopwords')

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Found GPU at: /device:GPU:0


In [0]:
!cp drive/My\ Drive/ikea.csv .

In [0]:
import numpy as np
from numpy import array
import pandas as pd
from random import randint
from pickle import dump, load
from sklearn.model_selection import train_test_split
import sys
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


def clean_text(input):
  # lowercase everything to standardize it
  input = input.lower()

  # tokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)
  
  # remove punctuation
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]

  # remove non alphabetic 
  tokens = [word for word in tokens if word.isalpha()]
  
	# make lower case
  tokens = [word.lower() for word in tokens]
  return tokens


def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# save tokens to file, one sequence per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

  
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [0]:
ikea_items = pd.read_csv('ikea.csv')

# some items do not have descriptions from the specific box
ikea_items = ikea_items.dropna()

# some descriptions are identical
desc_uni = ikea_items.drop_duplicates(subset='description')

# average description length for future generation
# desc_avg = round(sum( map(len, desc_uni) ) / len(desc_uni))
# desc_std = map(len, desc_uni).std()

# split train and test
desc_train, desc_test = train_test_split(desc_uni, test_size=0.2)
pd.DataFrame(desc_train).to_csv('ikea_train.csv')
pd.DataFrame(desc_test).to_csv('ikea_test.csv')

# make one corpus
desc_single = ' '.join(desc_train.description)

In [0]:
!cp ikea_test.csv drive/My\ Drive/.
!cp ikea_train.csv drive/My\ Drive/.

In [0]:
tokens = clean_text(desc_single)

In [111]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

Total Tokens: 87190
Unique Tokens: 3501


In [0]:
# make sequences of words from the full corpus
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
  
#print('Total Sequences: %d' % len(sequences))

out_filename = 'ikea_train_sequences.txt'
save_doc(sequences, out_filename)

In [0]:
!cp ikea_train_sequences.txt drive/My\ Drive/.

In [0]:
# load
in_filename = 'ikea_train_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [0]:
# code as integers

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [0]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [0]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [0]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
# print(model.summary())

In [0]:
# compile model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

# helpful checkpoints
filepath = "word_model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
desired_callbacks = [checkpoint]

In [96]:
# fit model
model.fit(X, y, epochs=10, batch_size=256, callbacks=desired_callbacks)

Epoch 1/10
86431/86431 [==============================] - 52s 606us/step - loss: 4.5034 - acc: 0.2070

Epoch 00001: loss improved from 4.95669 to 4.50337, saving model to word_model_weights_saved.hdf5
Epoch 2/10
86431/86431 [==============================] - 52s 606us/step - loss: 4.3225 - acc: 0.2239

Epoch 00002: loss improved from 4.50337 to 4.32249, saving model to word_model_weights_saved.hdf5
Epoch 3/10
86431/86431 [==============================] - 52s 604us/step - loss: 4.1605 - acc: 0.2405

Epoch 00003: loss improved from 4.32249 to 4.16053, saving model to word_model_weights_saved.hdf5
Epoch 4/10
86431/86431 [==============================] - 52s 603us/step - loss: 4.0103 - acc: 0.2562

Epoch 00004: loss improved from 4.16053 to 4.01032, saving model to word_model_weights_saved.hdf5
Epoch 5/10
86431/86431 [==============================] - 52s 602us/step - loss: 3.8585 - acc: 0.2740

Epoch 00005: loss improved from 4.01032 to 3.85851, saving model to word_model_weights_saved.

In [0]:
# save the model
model.save('ikea_word_model.h5')
# save the tokenizer
dump(tokenizer, open('word_tokenizer.pkl', 'wb'))

In [0]:
!cp ikea_word_model.h5 drive/My\ Drive/.
!cp word_tokenizer.pkl drive/My\ Drive/.
!cp word_model_weights_saved.hdf5 drive/My\ Drive/.

In [0]:
## load cleaned text sequences
#in_filename = 'ikea_train_sequences.txt'
#doc = load_doc(in_filename)
#lines = doc.split('\n')
#
#seq_length = len(lines[0].split()) - 1
#
## load the model
#model = load_model('ikea_word_model.h5')
#
## load the tokenizer
#tokenizer = load(open('word_tokenizer.pkl', 'rb'))
#
## select a seed text
#seed_text = lines[randint(0,len(lines))]
#print(seed_text + '\n')
#
## generate new text
#generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
#print(generated)